In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

import csv
import time
import re
import json
import pandas as pd
import numpy as np
from datetime import date, timedelta as td, datetime

from sklearn import base
from sklearn import cross_validation
from sklearn.metrics import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion

from scipy import stats

def printmetrics(y_predict,y_true):
    print "Abs Mean Error: ", metrics.mean_absolute_error(y_true, y_predict)
    print "R2 Score: ", metrics.r2_score(y_true, y_predict)
    print "RMSE: ", np.sqrt(metrics.mean_squared_error(y_true, y_predict))

def compute_error(clf, X, y):
    return - cross_validation.cross_val_score(clf, X, y, cv=cv, scoring='mean_squared_error').mean()

def side_by_side (*objs, **kwds):
    from pandas.core.common import adjoin
    space = kwds.get('space',4)
    reprs = [repr(obj).split('\n') for obj in objs]
    print adjoin(space,*reprs)

def unix_time(dt):
    epoch = datetime.utcfromtimestamp(0)
    delta = dt - epoch
    
    if isinstance(delta,pd.Series):
        return delta.astype('timedelta64[s]')
    else:
        return delta.total_seconds()

def unix_time_millis(dt):
    return unix_time(dt)*1000


# input is a pandas series
def year_frac(pd_series):
    return pd_series.apply(lambda x: x.year + float((x-datetime(x.year,1,1) ).days) / \
                           ( datetime(x.year+1,1,1)-datetime(x.year,1,1) ).days )

### Goal: Predicting brand-to-generic conversion time
* Import table from SQL
* Combine the EPC and dosage into the same column
* Create pipeline for combining the word features and the date feature.
* Use RidgeCV for linear regression
  * The CV avoids overfitting, especially on some less common pharmaceutical classes


In [ ]:
class WordXFormer(base.BaseEstimator, base.TransformerMixin):
        
    def __init__(self ):
        # init count vectorizer. This is the fitting 
        self.cv = CountVectorizer()
#         self.cv = CountVectorizer(vocabulary=dict(zip(vocab,xrange(len(vocab)))))
            
    def fit(self, X, y):
        
        return self

    def transform(self, X):
        
        # This sets the sparse matrix for all words to be counted.
        return self.cv.fit_transform( X['epc_dosageform'] ).toarray()

class DateXFormer(base.BaseEstimator, base.TransformerMixin):
        
    def fit(self, X, y):
        
        return self

    def transform(self, X):
        
        # Actually return year and month separately.
        # Possible to try with a "julian" date as well.
        return list(zip(list(X['bxdate'].apply(lambda x: x.year)),list(X['bxdate'].apply(lambda x: x.month))))

    
class LinearEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self):
        
        # Use a cross validated Ridge to avoid overfitting on particular words.
        self.lr = RidgeCV()

    def fit(self,X,y):
        self.lr.fit(X, y)
        return self

    def predict(self, X):
               
        return self.lr.predict(X)
    
    def score(self, X,y):
        
        return self.lr.score(X,y)

In [ ]:
# CSVs:
# firstbxgxmatch_fdalist - Grouped by the nonproprietaryname + dosage form as described by the Orange Book. bxdate - gxdate > 180.

df = pd.read_csv('firstbxgxmatch_fdalist.csv', sep = ',')
df['bxdatestr'] = df['bxdate']
df['gxdatestr'] = df['gxdate']
# Change to datetime format
df['bxdate'] = pd.to_datetime(df['bxdate'])
df['gxdate'] = pd.to_datetime(df['gxdate'])


df_epc = df[df['epc'].notnull() & (df['bxdate'] > '1995-01-01')]
df_epc['epc_dosageform'] = df_epc['epc'].map(str) + df_epc['dosageform'].map(str)
        
# Estimation of brand name date and generic name date for plot
bxdate = df['bxdate'].apply(lambda x: x.year) + \
        df['bxdate'].apply(lambda x: x.month)/12 + \
        df['bxdate'].apply(lambda x: x.year)/365
gxdate = df['gxdate'].apply(lambda x: x.year) + \
        df['gxdate'].apply(lambda x: x.month)/12 + \
        df['gxdate'].apply(lambda x: x.year)/365

        
plt.figure()
plt.title('Brand vs Generic Release Dates')
plt.xlabel('Brand Release Year')
plt.plot(bxdate.values,gxdate.values,'b.')
plt.ylabel('Generic Release Year')

In [ ]:
# Model creation and fitting

df_data = df_epc[['epc_dosageform','bxdate']]
y_true = (df_epc['bxgxtime'].apply(lambda x: float(x))).values
                   

# Feature union
feat_union = FeatureUnion([
    ('wordx', WordXFormer()),   # EPC and Dose Count Vectorizer
    ('datex', DateXFormer()),   # Date of Release
  ])
# Actual pipeline
bxtogx_pipeline = Pipeline([('features',feat_union), ('lr',LinearEstimator())])

# Fit model
bxtogx_pipeline.fit(df_data,y_true)

# Get prediction
y_predict = bxtogx_pipeline.predict(df_data)


print bxtogx_pipeline.score(df_data,y_true )

printmetrics(y_predict/365,y_true/365)

plt.plot(y_predict, y_true,'b.')
plt.plot(xrange(1,6000), xrange(1,6000),'k-')

In [ ]:
# Creation of scatterplot for NVD3

df_epc['predict'] = (y_predict)/365
df_epc['true'] = (y_true)/365

df_epc_after1995 = df_epc[( df_epc['bxdate'].apply(lambda x:x.year) > 1995 ) & ( df_epc['gxdate'].apply(lambda x:x.year) > 1995 )]
df_epc_after1995 = df_epc[df_epc['predict'] > 365/180]

json_obj = []
df_epc_after1995 = df_epc_after1995.reset_index()
print len(df_epc_after1995)

for i in np.arange(len(df_epc_after1995)):
        
    json_obj.append( {'bxname': df_epc_after1995['bxname'][i] ,
             'gxname': df_epc_after1995['gxname'][i] , 
             'epc': df_epc_after1995['epc'][i] , \
             'predict' : df_epc_after1995['predict'][i] , \
             'true' : df_epc_after1995['true'].loc[i] , \
             'bxdatestr' : df_epc_after1995['bxdatestr'][i] } )
    
    
with open('epcdosemodel.json', 'w') as outfile:
    json.dump(json_obj, outfile)

### Fourier Transform Frequency Analysis
* Create new dataframe where index is a range of dates
* Join SQL csv to dataframe, and perform FFT

In [ ]:
# Supplier trends. How likely are they to release stuff, and when?
# Look at the top suppliers first.
df_majorsuppliers = pd.read_csv('major_supplier_release_dates.csv', sep = ',')
df_majorsuppliers['startmktdate'] = pd.to_datetime(df_majorsuppliers['startmktdate'])

# Bar graph by startmktdate MONTH
years = [2009,2010,2011,2012,2013,2014, 2015]
fig = plt.figure()
ax = fig.add_subplot(111)

avg_dayseries =  df_majorsuppliers.groupby(df_majorsuppliers['startmktdate'] \
                                .apply(lambda x: ('0'+ str(x.month))[-2:] +  ('0'+ str(x.day))[-2:]))['prodid'].count()
avg_dayseries.plot()
plt.title('Time Series of Average Number of Gx Launches \n (Averaged between 2010-2014)')
plt.xlabel('Month-Day')

for year in years:
    df_majorsuppliers_year = df_majorsuppliers[df_majorsuppliers['startmktdate'].apply(lambda x: x.year == year)]
    oneyear_dayseries =  df_majorsuppliers_year.groupby(df_majorsuppliers['startmktdate'] \
                                    .apply(lambda x: ('0'+ str(x.month))[-2:] +  ('0'+ str(x.day))[-2:]))['prodid'].count()
    avg_dayseries = pd.concat([avg_dayseries, oneyear_dayseries], axis=1)

avg_dayseries.columns = ['avg','2009','2010','2011','2012','2013','2014','2015']
avg_dayseries = avg_dayseries.fillna(0)

# Split becasue this groupby takes a while
s_releases = df_majorsuppliers.sort(['startmktdate']).groupby(['startmktdate'])['startmktdate'].count()
# General statistics:
s_releases_after_2k9 = s_releases[s_releases.index.year >= 2009]
print "MEAN: ", np.mean(s_releases_after_2k9)
print "25%: ", np.percentile(s_releases_after_2k9,25)
print "50%: ", np.percentile(s_releases_after_2k9,50)
print "75%: ", np.percentile(s_releases_after_2k9,75)
print "95%: ", np.percentile(s_releases_after_2k9,95)
print "97.5%: ", np.percentile(s_releases_after_2k9,97.5)

In [ ]:
# Plotting the fourier transform of all the releases

start = date(2009, 1, 1)
finish = date(2015, 6, 30)
rng = pd.date_range(start,finish)
df_majorsuppliers_ts = pd.DataFrame(index = rng)

# Count the number of releases every day
df_eachday =  df_majorsuppliers[df_majorsuppliers['startmktdate'] >= datetime(2009,1,1)].groupby(['startmktdate'])['startmktdate'].count()
df_majorsuppliers_ts = df_majorsuppliers_ts.join(df_eachday).fillna(0)

fs = 365 # Sampling rate in samples per year
fft = np.fft.fft(df_majorsuppliers_ts.values,axis=0)
xVals = fs * np.arange(float(len(fft)))/ len(fft) # Get frequency values for each of the fft values

plt.plot(xVals, np.abs(fft))
plt.ylim([0, 6E3]) # Zoom in on y axis
plt.xlim([0, max(xVals)/2]) # Zoom in on x axis
#Note: largest meaningful x axis value is max(xVals)/2
plt.title('Frequency Analysis of Gx Launches Over 2010-2014')
plt.xlabel('Frequency Of Releases')
plt.ylabel('Magnitude')
plt.show()

# Local maxima:
print np.nonzero(np.abs(fft) > 3000)
print xVals[339], xVals[2033]

# Performing PCA on supplier monthly releases

In [ ]:
df_monthly_releases_11to14 = pd.read_csv('monthly_releases_2011to2014.csv', sep = ',')
yrs = df_monthly_releases_11to14['yr'].unique()
mos = df_monthly_releases_11to14['mo'].unique()
supplier = df_monthly_releases_11to14['supplier'].unique()

df_monthly_index = pd.DataFrame(index=sorted(mos))
df_yearly_release = pd.DataFrame(index=sorted(mos))

# Group into dataframe where columns are YEAR + SUPPLIER and rows are MONTH (JAN-DEC)
for supply in supplier:
    for yr in yrs:
        colname = str(yr) + ' ' + str(supply)
        
        df_tmp = df_monthly_releases_11to14[ (df_monthly_releases_11to14['yr'] == yr) & \
                                   (df_monthly_releases_11to14.supplier.apply(lambda x: x is supply)) ]

        
        
        if df_tmp['count'].sum() > 10:
            df_tmp.index = df_tmp['mo']
            df_yearly_release[colname] = pd.concat([df_monthly_index,df_tmp['count']],axis=1)        

df_yearly_release = df_yearly_release.fillna(0) 
df_yearly_release

In [ ]:
# Preparing matrices for PCA by making them zero-mean

mat = df_yearly_release.as_matrix().transpose()
shp = mat.shape
print (shp)
print(mat.mean(axis=1).shape)
# Have a 0 mean matrix
mat_0u = mat - np.tile(mat.mean(axis=1), (shp[1],1)).transpose()

# Also have a normalized matrix. This KILLS most of your variance, which makes a lot of sense. It should haha.
mat_norm = np.multiply(mat,np.tile( (np.divide(1,mat.sum(axis=1))), (shp[1],1) ).transpose())
mat_norm_0u = mat_norm - np.tile(mat_norm.mean(axis=1), (shp[1],1)).transpose()


In [ ]:
# Perform PCA, and get 12 components out. 
pca = PCA(n_components=12)
X_r = pca.fit(mat_0u).transform(mat_0u)


print('explained variance ratio (first 12 components): %s'
      % str(pca.explained_variance_ratio_))
print

# Print first four components. 
print(np.round( pca.components_[:4]*1000)/1000)

### Rebuild all the components but using the eigenvectors/components as determined by the PCA by solving the formula:



$$ COL = \left[ \begin{array}{ccc} 
PCA_1 \\
\vdots\\
\vdots\\
\end{array} \right]  *  
w_1 + 
\left[ \begin{array}{ccc} 
PCA_2 \\
\vdots\\
\vdots\\
\end{array} \right]  *  
w_2 + 
\left[ \begin{array}{ccc} 
PCA_3 \\
\vdots\\
\vdots\\
\end{array} \right]  *  
w_3 + ...$$

### where COL is the supplier-year 12-column vector, PCA$_1$,$_2$,$_3$ are the principal components, and w$_1$,$_2$,$_3$ are the weights of the eigenvectors.

In [ ]:
components=[]
suppliers = df_yearly_release.columns

for num,row in enumerate(X_r):
    tmp = np.dot(np.linalg.inv(pca.components_),np.matrix(row).T)
    tmp = np.asarray(tmp).reshape(-1).tolist()
    tmp.append(suppliers[num])
    components.append(tuple( tmp ))

In [ ]:
# First components, top 10 suppliers that match
com = 0

sorted_components = sorted(components, key=lambda x: x[com], reverse=True)
print (np.round(pca.components_[com]*100)/100)
top10 = []
bot10 = []
weight = []
for j in xrange(15):
    weight.append( ( sorted_components[j][com], sorted_components[-j-1][com] ) )
    top10.append(sorted_components[j][-1])
    bot10.append(sorted_components[-j -1][-1])

print (top10)
print (bot10)
# Uncomment to see feature vectors that are the top10/bot10
# print (df_yearly_release[top10].as_matrix().transpose())
# print (df_yearly_release[bot10].as_matrix().transpose())

#### Notes on PCA
* Since all components are orthogonal, it's probably good at representing the model after the second component
* It also "averages" clusters to find the "major" component
* A K-Nearest-Neighbors clustering algorithm is much more appropriate/intuitive at "grouping" the suppliers together
* Below is a rough sample on how a K-Means clustering algorithm could work

In [ ]:
# Preliminary test on K-Means

from sklearn.cluster import KMeans
from collections import Counter

mat = df_yearly_release.as_matrix().transpose()
suppliers = df_yearly_release.columns

KMeanEstimator = KMeans(n_clusters = 30, n_init=20, max_iter =1000)
KMeanEstimator.fit(mat)
labels = KMeanEstimator.labels_
print labels
print(mat.shape)


supplier_clusters = [[]] * 30

for num,label in enumerate(labels):
    
    if not supplier_clusters[label]:
        supplier_clusters[label] = [suppliers[num][5:]]
    else:
        supplier_clusters[label].append(suppliers[num][5:])
        
# Count the number of times the same supplier appears in a cluster. Since they are grouped by years
# a supplier can appear at MOST 4 times in a cluster.
occur_twice = []
occur_three = []
occur_four = []

for supplier_cluster in supplier_clusters:
    dict_test = Counter(supplier_cluster)
    tmp = [x for num,x in enumerate(dict_test.keys()) if dict_test.values()[num] == 3]
    if tmp:
        tmp.append(np.round(KMeanEstimator.cluster_centers_[labels[num]]*1000/1000).tolist())
        occur_three.append(tmp)

print occur_three


### (Failed) Attempt to model suppliers

####Features
+ totsuppliers: Number of suppliers that release the brand name drug before the first ANDA is released
+ totreleases: Number of brand names (as counted by unique NDCs) released
+ julian_firstmktdate: The date the brand name was released (in julian format)
+ Time for bxdate/gxdate conversion

####To Predict:
+ Number of generic suppliers 1 year after the release

####Notes:
* Preliminary model. Did not do much tweaking
* Possible to improve if there were sales data on each brand name drug
* Other possible features to help improve model: Drug type, dosage form/route, etc

In [ ]:
print df_supplier_model.columns

In [ ]:
df_supplier_model = pd.read_csv('supplier_model.csv', sep = ',')
df_supplier_model['td_firstmktdate'] = pd.to_datetime(df_supplier_model['firstmktdate'])
df_supplier_model['julian_firstmktdate'] = df_supplier_model.td_firstmktdate.apply(lambda x: x.to_julian_date())

# Xfeatures: totsuppliers, totreleases, firstmktdate, bxgxdiff
# YModel: nsuppliers
lr = LinearRegression()
df_Xfeatures = df_supplier_model[['totsuppliers','totreleases','julian_firstmktdate', 'bxgxdiff']]
lr.fit(df_Xfeatures, df_supplier_model['nsuppliers'])
y_true = df_supplier_model['nsuppliers']
y_predict = lr.predict(df_Xfeatures)

# Essentially there is relationship. 
printmetrics(y_true,y_predict)
plt.plot(y_true,y_predict,'b.')
